In [1]:
import cv2
import numpy as np

class VideoStitcher:
    def __init__(self):
        self.cachedH = None

    def stitch_frames(self, frameA, frameB):
        if self.cachedH is None:
            (kpsA, featuresA) = self.detect_and_describe(frameA)
            (kpsB, featuresB) = self.detect_and_describe(frameB)
            matches = self.match_keypoints(kpsA, kpsB, featuresA, featuresB)

            # Check if there are enough matches to create a panorama
            if matches is None or len(matches) < 2:
                return None

            # Cache the homography matrix
            self.cachedH = self.compute_homography(matches, kpsA, kpsB)

        # Apply a perspective transform to stitch the frames together
        result = cv2.warpPerspective(frameA, self.cachedH, (frameA.shape[1] + frameB.shape[1], frameA.shape[0]))

        # Combine the frames by blending them
        result[:, 0:frameB.shape[1]] = frameB

        return result

    def detect_and_describe(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Use ORB for keypoint detection and description
        orb = cv2.ORB_create()
        (kps, features) = orb.detectAndCompute(gray, None)
        kps = np.float32([kp.pt for kp in kps])

        return kps, features

    def match_keypoints(self, kpsA, kpsB, featuresA, featuresB):
        matcher = cv2.BFMatcher()
        raw_matches = matcher.knnMatch(featuresA, featuresB, 2)

        # Apply ratio test
        matches = []
        for m in raw_matches:
            if len(m) == 2 and m[0].distance < m[1].distance * 0.75:
                matches.append((m[0].trainIdx, m[0].queryIdx))

        # Check if there are enough matches
        if len(matches) < 2:
            return None

        return matches

    def compute_homography(self, matches, kpsA, kpsB):
        ptsA = np.float32([kpsA[i] for (_, i) in matches])
        ptsB = np.float32([kpsB[i] for (i, _) in matches])

        H, _ = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, 4.0)
        return H


In [2]:
def stitch_videos(video_source1, video_source2, output_video_path):
    cap1 = cv2.VideoCapture(video_source1)
    cap2 = cv2.VideoCapture(video_source2)

    _, first_frame1 = cap1.read()
    _, first_frame2 = cap2.read()

    video_stitcher = VideoStitcher()

    # Get video frame dimensions
    height, width, _ = first_frame1.shape
    output_video = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (width * 2, height))

    while True:
        ret, left = cap1.read()
        ret1, right = cap2.read()

        if not ret or not ret1:
            break

        stitched_frame = video_stitcher.stitch_frames(left, right)

        if stitched_frame is not None:
            output_video.write(stitched_frame)
            cv2.imshow("Stitched Video", stitched_frame)
            if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' to exit
                break

    cap1.release()
    cap2.release()
    output_video.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_source1 = "Downloads/vid_6.mp4"
    video_source2 = "Downloads/vid_5.mp4"
    output_video_path = "output_video.mp4"

    stitch_videos(video_source1, video_source2, output_video_path)

OpenCV: Couldn't read video stream from file "Downloads/vid_6.mp4"
OpenCV: Couldn't read video stream from file "Downloads/vid_5.mp4"


AttributeError: 'NoneType' object has no attribute 'shape'